## 1. 현재까지의 연구 흐름
### 프로젝트 목표 : 유저 선호도가 모호하다. 유저의 정보가 없다를 대화형 추천 시스템으로 해결한 영화 추천 시스템을 만들겠다.
### 메인 베이스 논문 : 환자의 발화단계에 맞는 agent를 활용하여 필요에 맞는 기능 제공

### 현재 주차 목표 : 검색, 추천 두 단계 중 선택하여 실험하여 제공

## 2. 현재 주차에서의 연구 진행 아이디어
- 추천 받은 영화가 만족 -> 본인의 취향과 현재 상태(감정, 컨텐츠 소비 상태)에 추천받은 영화가 만족
- 사용자의 취향과 현재 상태를 추출하기 위한 선 발화가 필요 -> mbti 가 뭐야 오늘 기분은 어때 의 두 질문으로 선정
- 해당 질문 답변에 따른 다른 agent or 프롬프트 수정을 통해 추천을 위한 데이터 검색
- 예시) 특정 mbti의 사람이 특정 상태일때 어떤 특징의 영화가 좋읗지
- target user - 한국인, 영화를 좋아하는 편(많은 영화를 봤다)
- 데이터 DB를 chromaDB에 저장
- self query Retriever 적용해보기

## 00 패키지 import

In [1]:
import pandas as pd
import os

import chromadb
from langchain.chat_models import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain_core.retrievers import BaseRetriever
from langchain import LLMChain, PromptTemplate

from dotenv import load_dotenv

load_dotenv()

True

## 01 엑셀데이터 chromaDB 적재

In [2]:

df = pd.read_csv("../../preprocessed_movie_sample.csv")
df.head()

,title,director,screenwriter,plot,rating,rating_count,actors,comment_ratings,comment_texts,genres,countries,running_time,adult
0,월플라워,['스티븐 크보스키'],['스티븐 크보스키'],말 못할 트라우마를 가지고 자신만의 세계에 갇혀있던 ‘찰리’는 고등학교 신입생이 돼...,3.8,305000.0,"['에즈라 밀러', '로건 메먼', '엠마 왓슨']","['4.0', '5.0', '4.0', '2.0', '4.0', '4.0', '3....","['그 터널을 무사히 빠져나온 모두에게', '사람은 자신의 크기에 맞는 사랑을 한다...","['드라마', '로맨스']",['미국'],102.0,0.0
1,기생충,['봉준호'],"['봉준호', '한진원']",“폐 끼치고 싶진 않았어요” 전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호)...,4.3,1265000.0,"['박소담', '송강호', '조여정', '최우식', '이선균', '장혜진', '이정은']","['5.0', '4.5', '5.0', '0.5', '4.5', '5.0', '4....","['황금종려상 받은 영화를 자막없이 볼 수 있는 행복 190529', '상승과 하강...",['드라마'],['한국'],131.0,0.0
2,메멘토,['크리스토퍼 놀란'],['크리스토퍼 놀란'],"아내가 살해당한 후, 10분밖에 기억 못하는 단기기억상실증에 걸린 남자가 사진, 메...",4.1,641000.0,"['조 판톨리아노', '가이 피어스', '캐리 앤 모스']","['5.0', '5.0', '4.5', '5.0', '5.0', '5.0', '5....",['사람은 보고싶은 것만 보고 듣고싶은 것만 듣고 믿고 싶은 것만 믿고 기억하고 싶...,"['미스터리', '스릴러']",['미국'],113.0,0.0
3,노인을 위한 나라는 없다,"['에단 코엔', '조엘 코엔']","['에단 코엔', '조엘 코엔']",르웰린 모스(조쉬 브롤린)는 총격전이 벌어진 끔찍한 현장에서 우연히 이백만 달러가 ...,4.0,349000.0,"['토미 리 존스', '하비에르 바르뎀', '조쉬 브롤린']","['5.0', '5.0', '5.0', '5.0', '3.5', '5.0', '5....","['그거 아세요? 이 영화에는 배경음악이 단 일초도 쓰이지 않았다는것', '피튀기는...","['범죄', '드라마', '스릴러']",['미국'],122.0,1.0
4,파이트 클럽,['데이비드 핀처'],['짐 유힐'],당신이 알고 있는 모든 것은 허구다! 비싼 가구들로 집 안을 채우지만 삶에 강한 공...,4.1,371000.0,"['브래드 피트', '에드워드 노튼', '헬레나 본햄 카터']","['5.0', '5.0', '5.0', '4.5', '4.0', '5.0', '4....","['현대문명의 허상을 조롱하는 통렬한 블랙코미디.', '영화 자체가 섹시하다', '...","['드라마', '액션']","['미국', '독일', '이탈리아']",139.0,1.0


In [3]:
# ChromaDB 클라이언트 초기화
client = chromadb.Client()

# 영화 데이터를 위한 컬렉션 생성
collection = client.create_collection("movies")

In [4]:
df.columns

Index(['title', 'director', 'screenwriter', 'plot', 'rating', 'rating_count',
       'actors', 'comment_ratings', 'comment_texts', 'genres', 'countries',
       'running_time', 'adult'],
      dtype='object')

In [5]:
model_name = "BAAI/bge-m3"
model_kwargs = {'device': 'mps'}
encode_kwargs = {'normalize_embeddings': True}
huggingface_ef = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/var/folders/dn/qq5pnzjj1c98r1dbp_pgns1w0000gn/T/ipykernel_50432/1850178229.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_ef = HuggingFaceEmbeddings(
/Users/visuworks/Documents/00_project/SmallTalk2Rec_Exp/my_env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [6]:
# 임베딩 생성 
df['plot_embedding'] = df['plot'].apply(lambda text: huggingface_ef.embed_query(text))

### comment 점수와 코멘트로 새로운 형태로 생성 필요


In [7]:
for index, row in df.iterrows():
    collection.add(
        documents=[row['plot']],  # 필드별 정보
        embeddings=[row['plot_embedding']],  # 필드별 임베딩
        metadatas=[{
            'title': row['title'],
            'director': row['director'],
            'screenwriter' : row['screenwriter'],
            'rating': row['rating'],
            'actors': row['actors'],
            'genres': row['genres'],
            'countries' : row['countries'],
            'running_time' : row['running_time'],
            'adult' : row['adult']
        }],
        ids=[str(index)]  # 고유 ID 설정
    )

In [8]:
### 사용 실패 - collection 할당이 인식이 안됨
class ChromaDBRetriever(BaseRetriever):
    def __init__(self, collection):
        # 생성자에서 전달된 collection을 클래스 변수에 할당
        self.collection = collection

    # query를 받아서 ChromaDB에서 관련 문서들을 검색한 후 Document 리스트로 변환
    def get_relevant_documents(self, query: str):
        # ChromaDB에서 쿼리와 유사한 문서 검색
        results = self.collection.query(
            query_texts=[query],  # 사용자가 입력한 검색 쿼리
            n_results=5  # 상위 5개의 결과 가져오기
        )
        
        # 검색 결과에서 Document 객체로 변환하여 반환
        documents = []
        for i in range(len(results['documents'][0])):
            doc_content = results['documents'][0][i]
            metadata = {
                '제목': results['metadatas'][0][i]['제목'],
                '감독': results['metadatas'][0][i]['감독'],
                '배우': results['metadatas'][0][i]['배우'],
                '평점': results['metadatas'][0][i]['평점'],
                '리뷰사이트 평점': results['metadatas'][0][i]['리뷰사이트 평점'],
                '평점수': results['metadatas'][0][i]['평점수'],
            }
            documents.append(Document(page_content=doc_content, metadata=metadata))
        
        return documents

/var/folders/dn/qq5pnzjj1c98r1dbp_pgns1w0000gn/T/ipykernel_50432/330892639.py:2: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class ChromaDBRetriever(BaseRetriever):


In [9]:
vectorstore = Chroma(
    collection_name="movies",
    client=client,
    embedding_function=huggingface_ef
)

/var/folders/dn/qq5pnzjj1c98r1dbp_pgns1w0000gn/T/ipykernel_50432/2464781284.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [10]:
# Openai에서 사용할 LLM 모델
llm = ChatOpenAI(temperature=0, 
               model='gpt-4o-mini-2024-07-18')


# RAG 설정: OpenAI LLM을 사용한 추천 생성
rag = RetrievalQA.from_chain_type(
    llm=llm,  # OpenAI LLM 사용
    retriever=vectorstore.as_retriever(),  # ChromaDB에서 검색한 결과를 사용
    return_source_documents=True
)

# 사용자 입력을 받아 영화 추천 생성
def generate_movie_recommendation(user_query):
    result = rag({"query": user_query})
    return result["result"], result["source_documents"]

# 사용자 쿼리 입력 예시
user_query = "오늘 우울한데 어떤 영화를 보면 좋을까"
recommendation, source_docs = generate_movie_recommendation(user_query)

# 추천 결과 출력
print("Recommended Movies: ", recommendation)
print("Source Documents: ", source_docs)

/var/folders/dn/qq5pnzjj1c98r1dbp_pgns1w0000gn/T/ipykernel_50432/97496504.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0,
/var/folders/dn/qq5pnzjj1c98r1dbp_pgns1w0000gn/T/ipykernel_50432/97496504.py:15: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = rag({"query": user_query})


Recommended Movies:  우울할 때는 감정에 공감할 수 있는 영화나 따뜻한 메시지를 전하는 영화를 보는 것이 좋습니다. 예를 들어, <거인>은 인간의 다층적인 연약함을 보여주면서도 공존의 가능성을 고민하는 이야기로, 감정적으로 깊이 있는 경험을 제공할 수 있습니다. 또한, <엘리멘트 시티>는 다양한 원소들이 살아가는 세계에서의 특별한 우정과 새로운 경험을 다루고 있어, 긍정적인 감정을 불러일으킬 수 있을 것입니다. 이 두 영화를 고려해 보세요.
Source Documents:  [Document(metadata={'actors': "['티모시 샬라메', '엘 패닝', '']", 'adult': 0.0, 'countries': "['미국']", 'director': "['우디 앨런']", 'genres': "['코미디', '로맨스']", 'rating': 3.3, 'running_time': 92.0, 'screenwriter': "['우디 앨런']", 'title': '레이니 데이 인 뉴욕'}, page_content='상상해 봐요 막 떨어지기 시작한 빗방울 센트럴 파크 델라코트 시계 아래 누군가 당신을 기다리고 있다면... 재즈를 사랑하는 ‘개츠비’(티모시 샬라메) 영화에 푹 빠진 ‘애슐리’(엘르 패닝) 뉴욕이 좋은 ‘챈’(셀레나 고메즈) 매력적인 세 남녀가 선사하는 로맨틱 해프닝!'), Document(metadata={'actors': "['엠마 스톤', '라이언 고슬링', '스티브 카렐']", 'adult': 0.0, 'countries': "['미국']", 'director': "['존 레쿼', '글렌 피카라']", 'genres': "['로맨틱 코미디', '드라마', '코미디']", 'rating': 3.5, 'running_time': 118.0, 'screenwriter': "['댄 포겔만']", 'title': '크레이지 스투피브 러브'}, page_content='좋은 직장, 좋은 집, 그리고 멋진 아이들까지 모든 것이 완벽하게 

## 사용자의 MBTI와 상태를 질문을 해서 해당

In [11]:
def user_condition(user_query):
    user_condition = ChatOpenAI(model= 'gpt-4o-mini', temperature= 0)

    usertag_prompt = """너는 MBTI와 감정상태를 기반으로 어떤 경험을 하는 것이 좋은지 추천하는 역활이야. 

    user: {query}
    answer: """

    usertag_template = PromptTemplate(
        template  = usertag_prompt,
        input_variables = ['query']
    )

    query_making = LLMChain(
        llm = user_condition,
        prompt = usertag_template
    )

    result = query_making.invoke(user_query)
    return result['text']


In [12]:
user_condition('내 MBTI는 ENTP이고 오늘은 우울해')

/var/folders/dn/qq5pnzjj1c98r1dbp_pgns1w0000gn/T/ipykernel_50432/28203293.py:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  query_making = LLMChain(


'우울한 기분을 느끼고 계시군요. ENTP 유형은 창의적이고 호기심이 많으며, 새로운 아이디어와 경험을 추구하는 경향이 있습니다. 이런 기분을 조금이나마 덜어줄 수 있는 몇 가지 활동을 추천해드릴게요.\n\n1. **창의적인 프로젝트 시작하기**: 새로운 아이디어를 떠올려보세요. 그림 그리기, 글쓰기, 또는 DIY 프로젝트 같은 창의적인 활동이 기분 전환에 도움이 될 수 있습니다.\n\n2. **토론이나 대화**: 친구나 가족과의 깊이 있는 대화를 통해 감정을 나누어 보세요. ENTP는 대화를 통해 생각을 정리하고 감정을 표현하는 데 도움이 될 수 있습니다.\n\n3. **산책이나 운동**: 자연 속에서의 산책이나 가벼운 운동은 기분을 전환하는 데 큰 도움이 됩니다. 신선한 공기를 마시며 생각을 정리해보세요.\n\n4. **새로운 정보 탐색**: 흥미로운 주제에 대해 공부하거나 다큐멘터리를 시청해보세요. 새로운 지식을 얻는 것이 기분을 좋게 할 수 있습니다.\n\n5. **유머와 웃음**: 코미디 영화나 유튜브 영상을 보며 웃어보세요. 유머는 기분을 전환하는 데 큰 힘이 됩니다.\n\n이런 활동들이 도움이 되길 바라며, 기분이 나아지길 바랍니다!'

## self query 리트리버 설정

In [14]:
from langchain.chains.query_constructor.base import AttributeInfo


# 메타데이터 필드 정보 생성
metadata_field_info = [
    AttributeInfo(
        name="title",
        description="The name of movie",
        type="string",
    ),
    AttributeInfo(
        name="director",
        description="The main diretor of movie",
        type="string",
    ),
    AttributeInfo(
        name="screenwriter",
        description="The main screenwriter of movie",
        type="string",
    ),
        AttributeInfo(
        name="rating",
        description="The score of movie",
        type="float",
    ),
            AttributeInfo(
        name="rating",
        description="The score of movie",
        type="float",
    ),
]

In [15]:
from langchain.retrievers.self_query.base import SelfQueryRetriever

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# SelfQueryRetriever 생성
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="story of movie",
    metadata_field_info=metadata_field_info,
)

In [16]:
retriever.invoke("평점이 4.8 이상인 제품을 추천해주세요")


[]